# Simulation of 1D Ising Model

## The method is from paper 2004 1D Ising Model

### Method 1: Combination of Classical and Quantum Algorithm

In [3]:
import numpy as np
import random
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

In [9]:
# Set Parameters
J = 1 # Coupling Strength
T = 1 # Temperature

# Calculate the Possibility
P = np.exp(-4 * J / T)

# Initialize the spins. This might be able to performed through quantum algorithms.
n = 5
spins = [1, 1, -1, 1, 1]

# Initialize the circuit
q = QuantumRegister(5)
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)

"""
For every loop, the program takes 3 spins from position i-1, i, and i+1 of the spins list. The program will then apply the Ising 
interaction on the spin at position i. After the ising interaction, the update spins is stored back to the spins list. The same 
process then repeated. If we don't consider the 1-dimensional ising model to be arranged in a circular shape (that is, the head spin
is not in neighbour with the tail spin), the program will apply ising interaction on all the spins except the first one and the last one.
"""


"""
In the following code, q[0] represents the spin i, q[1] represents the spins i-1, q[2] represents the spins i+1,
q[3] stands for the scratch qubit which will always be |0>, and q[4] represents the possibility to flip if change in energy is positive.
"""
for i in range (1, len(spins) - 1):
    # Set the scratch and possibility qubits
    qc.reset(q[3])
    qc.reset(q[4])
    qc.u(2 * np.arccos(np.sqrt(1 - P)), 0, 0, q[4])
    
    #Set up the qubits
    if spins[i] == 1:
        qc.x(q[0])
    else:
        qc.reset(q[0])
    
    if spins[i - 1] == 1:
        qc.x(q[1])
    else:
        qc.reset(q[1])
    
    if spins[i + 1] == 1:
        qc.x(q[2])
    else:
        qc.reset(q[2])
    
    # Apply Ising Interaction (In a 'conservative' way, the code can be modified to be more efficient)
    # Operation 1
    qc.mcx([q[0], q[1], q[2]], q[3]) 
    
    #Operation 2
    for j in range(3):
        qc.x(q[j])
    qc.mcx([q[0], q[1], q[2]], q[3])
    for j in range(3):
        qc.x(q[j])
    
    # Operation 3
    qc.cx(q[3], q[0])
    
    # Operation 4
    qc.ccx(q[4], q[3], q[0])
    
    # Measurement
    qc.measure(q[0], c)
    backend = Aer.get_backend('qasm_simulator')
    shots = 1024
    results = execute(qc, backend, shots=shots).result()
    counts = results.get_counts()
    
    # Update spin i
    if len(counts.keys()) == 1:
        newstat = int(list(counts.keys())[0])
    else:
        if list(counts.values())[0] > list(counts.values())[1]:
            newstat = int(list(counts.keys())[0])
        else:
            newstat = int(list(counts.keys())[1])
    if newstat == 1:
        spins[i] = 1
    elif newstat == 0:
        spins[i] = -1

print(spins)

[1, 1, -1, 1, 1]
